In [11]:
from IPython.display import clear_output

import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor

In [12]:
installs = pd.read_csv('data/installs.csv.gzip', low_memory=False, compression='gzip', parse_dates = ['created'], index_col=['created'])
installs = installs.sort_index()

In [13]:
ventana_1 = installs['2019-04-18':'2019-04-20'].reset_index()

In [14]:
train_ventana_2 = installs['2019-04-21':'2019-04-23'].reset_index().groupby('ref_hash').agg({'created' : 'min'}).reset_index()

In [15]:
train_ventana_2['created'] = train_ventana_2['created'] - pd.Timestamp('2019-04-21 00:00:00')
train_ventana_2['created'] = train_ventana_2['created'].transform(lambda x: x.total_seconds())

In [16]:
train_ventana_2 = train_ventana_2.merge(ventana_1["ref_hash"].drop_duplicates(), how="right")

In [17]:
train_ventana_2 = train_ventana_2.sort_values(["ref_hash"]).reset_index(drop=True)

In [18]:
train_ventana_2["created"].fillna(60*60*24*3, inplace=True)
train_ventana_2

,ref_hash,created
0,41863526108385,259200.00
1,135153013040192,259200.00
2,186034136943920,259200.00
3,365882020742330,259200.00
4,519199987760489,259200.00
5,530786270564316,259200.00
6,558877640599287,259200.00
7,655267966876774,259200.00
8,848739744907305,259200.00
9,918552236966790,259200.00


In [19]:
train_vent_2 = train_ventana_2["created"]
hash_a_pred = train_ventana_2["ref_hash"]

In [20]:
ventana_1['created'] = ventana_1['created'] - pd.Timestamp('2019-04-18 00:00:00')
ventana_1['created'] = ventana_1['created'].transform(lambda x: x.total_seconds())


In [21]:
ventana_1

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
0,1.560,70,1891515180541284343,4432995619177048534,NaN,False,False,6287817205707153877,NaN,NaN,Apsalar-Postback,NaN,NaN,NaN,NaN,NaN,8409380583076227678,NaN
1,1.851,70,1891515180541284343,5904733559638204455,NaN,False,False,6287817205707153877,NaN,NaN,Apsalar-Postback,NaN,NaN,NaN,NaN,NaN,806493034070542041,NaN
2,5.152,65,1891515180541284343,896373747754111825,NaN,False,True,6287817205707153877,3.083059e+17,5.274185e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 9; SM-G9650 Bu...,8c8af5e3-96e7-4a49-9f17-cafa7f300f2c,af_app_opened,False,NaN,6319394158544413414,6.977049e+18
3,5.589,27,1494519392962156891,3399210824535017892,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,trivago/216 CFNetwork/978.0.7 Darwin/18.5.0,NaN,NaN,True,NaN,4206683315615624585,5.221863e+18
4,6.795,339,1494519392962156891,1541425881979513687,NaN,False,False,6287817205707153877,NaN,6.794880e+18,http-kit/2.0,TikTok/109005 CFNetwork/758.5.3 Darwin/15.6.0,NaN,NaN,True,NaN,8335834010411202437,7.528974e+18
5,8.994,210,1891515180541284343,8942039642364169230,NaN,False,False,6287817205707153877,NaN,2.019322e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 8.0.0; FIG-LX3...,NaN,NaN,NaN,NaN,344839734532564260,3.301378e+18
6,9.158,65,1891515180541284343,896373747754111825,NaN,False,False,6287817205707153877,3.083059e+17,5.274185e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 9; SM-G9650 Bu...,NaN,NaN,False,NaN,6319394158544413414,6.977049e+18
7,11.221,65,1891515180541284343,6061394227238904841,NaN,False,True,6287817205707153877,2.208835e+18,4.444254e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 8.0.0; moto e5...,7158a6b0-3612-43d9-81ff-22ff562122fb,af_app_opened,False,NaN,8232896611563676069,6.977049e+18
8,12.695,117,1891515180541284343,681488082455723993,NaN,False,False,6287817205707153877,NaN,6.460612e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,3822778906095644506,3.301378e+18
9,12.865,121,1891515180541284343,2523223190175071839,NaN,False,True,6287817205707153877,2.523246e+18,2.827096e+17,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 8.1.0; LG-M700...,a301a579-2594-4834-9a1e-d0ec35cd89e7,Open,True,NaN,341785641840984326,6.977049e+18


In [22]:
ventana_1.drop(["click_hash", "event_uuid", "kind", "trans_id", "user_agent", "session_user_agent", "ref_type", "attributed", "device_countrycode", "device_brand", "ip_address", "device_language"],axis =1, inplace = True )

In [23]:
ventana_1 = ventana_1.fillna(ventana_1.mode().iloc[0])

In [24]:
ventana_1 = ventana_1.sort_values(["ref_hash", "created"]).reset_index(drop=True)

In [25]:
slideing = pd.DataFrame()
slideing["created"] = ventana_1["created"]
slideing['creac_sig'] = slideing["created"].shift(-1)


In [26]:
stop_points = ventana_1.groupby("ref_hash").last().reset_index()["created"]

In [27]:
j = 0
tiempo_min = 60*60*24*3
for i,row in slideing.iterrows():
    if stop_points[j] == row[0]:
        slideing.at[i, "creac_sig"] = tiempo_min
        j += 1

In [28]:
test_y = slideing["creac_sig"]

In [29]:
ventana_1["censored"] = 0

In [30]:
j = 0
for i,row in ventana_1.iterrows():
    if stop_points[j] == row[0]:
        ventana_1.at[i, "censored"] = 1

        j += 1

In [31]:
ultimos = ventana_1.groupby("ref_hash").last().reset_index()

In [32]:
ultimos["created"] = 60*60*24*3

**Encodeo las variables categoricas y strings**

In [33]:
"""le_usr_ag = LabelEncoder()
le_ss_usr_ag = LabelEncoder()
ventana_1['user_agent'] = le_usr_ag.fit_transform(ventana_1.user_agent)
ventana_1['session_user_agent'] = le_ss_usr_ag.fit_transform(ventana_1.session_user_agent)"""

"le_usr_ag = LabelEncoder()\nle_ss_usr_ag = LabelEncoder()\nventana_1['user_agent'] = le_usr_ag.fit_transform(ventana_1.user_agent)\nventana_1['session_user_agent'] = le_ss_usr_ag.fit_transform(ventana_1.session_user_agent)"

In [34]:
"""usr_ag_ohe = OneHotEncoder()
ss_usr_ag_ohe = OneHotEncoder()
X = usr_ag_ohe.fit_transform(ventana_1.user_agent.values.reshape(-1,1)).toarray()
Xm = ss_usr_ag_ohe.fit_transform(ventana_1.session_user_agent.values.reshape(-1,1)).toarray()"""

'usr_ag_ohe = OneHotEncoder()\nss_usr_ag_ohe = OneHotEncoder()\nX = usr_ag_ohe.fit_transform(ventana_1.user_agent.values.reshape(-1,1)).toarray()\nXm = ss_usr_ag_ohe.fit_transform(ventana_1.session_user_agent.values.reshape(-1,1)).toarray()'

X_train, X_test, y_train, y_test = \
    train_test_split(ventana_1, slideing["creac_sig"], test_size=0.2, random_state=769)

reg=linear_model.LinearRegression()

reg.fit(X_train,y_train)

pred=reg.predict(X_test)

mean_squared_error(y_test,pred)**0.5


In [36]:
reg=KNeighborsRegressor()
reg.fit(ventana_1, test_y)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform')

In [37]:
pred=reg.predict(ultimos)

### Tarda como 30 min o mas

In [ ]:
TIEMPO_A_CONSIDERAR = 60*60*24*3
def obtener_pred_corr(x_train, y_train, a_predecir, pred, aiuda):
    primera = True
    hay_menor = hay_menores(pred)
    k = 0
    while hay_menor:
        aiuda.append(k)
        train_a_agregar = []
        valores_a_agregar = pd.DataFrame(columns=[lb for lb in x_train])
        for i, valor in enumerate(pred):
            clear_output(wait=True)
            if valor <= TIEMPO_A_CONSIDERAR:
                    fila_a_agregar = a_predecir.iloc[i].copy()
                    usuario = fila_a_agregar["ref_hash"]
                    a_predecir.at[i, "created"] = valor
                    valores_a_agregar.loc[len(valores_a_agregar)] = fila_a_agregar
                    train_a_agregar.append(valor)
                    print(f"Current progress {np.round(i/132297 * 100, 2)}%")
        
        x_train = x_train.append(valores_a_agregar, ignore_index=True)
        y_train = y_train.append(train_a_agregar)
        
        reg.fit(x_train,y_train)
        pred=reg.predict(a_predecir)
        
        hay_menor = hay_menores(pred)
        primera = False
    return pred, ver_problem
      
             
def hay_menores(pred):
    for elem in pred:
        if elem <= TIEMPO_A_CONSIDERAR:
            return True
    return False

In [ ]:

# pred =obtener_pred_corr(ventana_1, test_y, ultimos, pred, mi_lis)


In [38]:
mean_squared_error(train_vent_2,pred)**0.5

39326.26535915615

In [39]:
pred

array([149596.0886, 253848.1338, 259200.    , ..., 186027.5566,
       259200.    , 259200.    ])

In [ ]:
mean_squared_error(train_vent_2,maximo)**0.5